# 1 컨텐츠 기반 필터링 실습_TMDB5000

* 콘텐츠 기반 필터링/ 아이템 기반 필터링
* 참고 데이터 : https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [49]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('../data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [50]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count',
                   'popularity', 'keywords', 'overview']]

In [51]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [52]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


### 텍스트 문자 1차 가공, 파이썬 딕셔너리를 리스트 형태로 변환

In [53]:
movies_df['genres'].head(1) #딕셔너리 형태

0    [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {...
Name: genres, dtype: object

In [54]:
# literal_eval : 문자 그대로 평가-파이썬 객체로 만들어줌
from ast import literal_eval #딕셔너리를 리스트 형태로 변환

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [55]:
movies_df['genres'] #파이썬 객체로 들어가 있는 상태

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2              [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
                                                       ...                                                 
4798            [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                       [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'...
4801                        

In [56]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name']for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name']for y in x])
movies_df[['genres','keywords']][:2]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship..."


### 장르 콘텐츠 필터링을 이용한 영화추천, 장르 문자열을 Count 벡터화 후에 코사인 유사도로 각 영활르 비교

### 장르 문자열 Count 기반 피처 벡터화 (몇행 몇열로 나타냄)

In [57]:
('*').join(['test','test2'])

'test*test2'

In [58]:
type(('*').join(['test', 'test2']))

str

### 문장의 유사도 측정하기
* 1. BOW(Bag of word)
    * 문서를 숫자 벡터로 변환하는 가장 기본적인 방법
    * min_df : 정수 또는 [0.0, 1.0]사이의 실수.디폴트1(단어장에 포함되기 위한 최소 빈도)
* 2. CountVectorizer : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만든다. Scikit-Learn의 문서 전처리 중 하나이다.
* 3. TF-IDF 
    * 단어의 빈도-역문서 빈도
    * term문서가 있을때 단어를 의미, TF-IDF(단어별 문서의 연관성 알고싶을때 사용)
    * TF(단어빈도) - IDF(중요도 반영 : Log(2/2)=0 두개의 문서 중 두개다 있어요)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer #0,1로~

#CountVectorizer를 적용하기 위해 공백 문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ('').join(x))

#min_df=0 : 단어장에 포함되기 위한 최소 빈도(단어장에 몇번 나와야지 쓸것이다)
#ngram_range=(1,2)=>ngram_range(min_n, max_n) 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))#'go home', 'very good'
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 1394)


In [60]:
movies_df['genres_literal']
#4083개 레코드 276개의 개별 단어 피처로 구성된 피처 벡터 행렬

0       ActionAdventureFantasyScience Fiction
1                      AdventureFantasyAction
2                        ActionAdventureCrime
3                    ActionCrimeDramaThriller
4              ActionAdventureScience Fiction
                        ...                  
4798                      ActionCrimeThriller
4799                            ComedyRomance
4800               ComedyDramaRomanceTV Movie
4801                                         
4802                              Documentary
Name: genres_literal, Length: 4803, dtype: object

### 장르에 따른 영화별 코사인 유사도 추출

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:4])

(4803, 4803)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [65]:
a = np.array([1.5, 0.2, 4.2, 2.5])
print(a)
print('인덱스 정렬:', a.argsort()) #가장 작은값부터의 순서는 첫번째(0.2) , 영번째(1.5), 두번째..
print('역순정렬',a[::-1])#그것의 역순으로 정렬한 값

[1.5 0.2 4.2 2.5]
인덱스 정렬: [1 0 3 2]
역순정렬 [2.5 4.2 0.2 1.5]


In [64]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0  870   46 ... 3172 3173 2401]]


### 특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성

In [70]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movie_df df에서 title컬럼이 입력된 title_name값인 df 추출
    title_movie = df[df['title']== title_name]
    
    #title_named을 가진 df index객체를 ndarray로 반환
    #sorted_ind인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로
    #top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    #추출된 top_n index들 출력, top_n index는 2차원 데이터 임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [72]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[3636  883 1149 3337 1464 1243 1847 3866 2582 4065]]


,title,vote_average
3636,Light Sleeper,5.7
883,Catch Me If You Can,7.7
1149,American Hustle,6.8
3337,The Godfather,8.4
1464,Black Water Transit,0.0
1243,Mean Streets,7.2
1847,GoodFellas,8.2
3866,City of God,8.1
2582,The Place Beyond the Pines,6.8
4065,Mi America,0.0


* 결과는 나오는데 유사성이 떨어져

### 평점이 높은 영화 정보 확인

In [73]:
movies_df[['title', 'vote_average', 'vote_count']].\
sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


* 한명이 10점을 주면 10점이 되 버린다. 신뢰가 없어.. stiff Upper Lips 같은 영화
* 영화 평점(vote)와 관련 평균평점(vote_average)-불공정
* 평점을 남김 count(vote_count)3개 다 5점
* vote 가 많을수록 평점이 5점이 나올수 없고, 떨어질수 있다.
* 참조:https://www.quora.com/How-does-IMDbs-rating-system-work

### 평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating)계산
    가중평점(Weighted Rating) = (v/(v+m) * R + (m/v+m)) *c **
    
* (1000/1000+300)*10 = 7.6
* (10/(10+30))*10 = 2.5

* v:개별 영화에 평점을 투표한 횟수
* m:평점을 부여하기 위한 최소 투표 횟수(ex.상위 60%, 40%)
    
* R:개별 영화에 대한 평균 평점
* C:전체 영화에 대한 평균 평점

In [75]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) #상위 60%
print('C:', round(C,3), 'm:', round(m,3))
#상위 60%는 370회 정도의 투표 횟수

C: 6.092 m: 370.2


In [77]:
percentile = 0.6
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(percentile)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v) * C)) #가중 평점

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [78]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].\
sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [93]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']== title_name]
    title_index = title_movie.index.values
    
    #top_n의 2배에 해당하는 장르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    #기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    #top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

In [95]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
1149,American Hustle,6.8,6.717525
2582,The Place Beyond the Pines,6.8,6.670483
1243,Mean Streets,7.2,6.626569
1370,21,6.5,6.413490
